<a href="https://colab.research.google.com/github/MuhammadBilall/Data-Science-Labs/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", delimiter=',', header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  rated = np.where(arr!=99)
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [47]:
arr = df.values
validationData, idx = replace(arr)
print(validationData)

[[ 74.    -7.82   8.79 ...  99.    99.    99.  ]
 [100.     4.08  -0.29 ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]


In [82]:
n_latent_factors = 5
user_ratings = df.values

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(latent_user_preferences.shape)
print(latent_item_features.shape)

(24983, 5)
(101, 5)


In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.00001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [0]:
sgd()

2262.6776684859205


In [73]:
user_ratings = validationData

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[23.80819488, 23.95901997, 22.6076254 , ..., 22.72168454,
        22.44137115, 22.32242787],
       [27.57873561, 24.95488638, 26.35258692, ..., 24.61968782,
        25.91270077, 26.24893134],
       [25.69622289, 25.00220687, 22.89279956, ..., 25.25620857,
        22.96886513, 25.93440603],
       ...,
       [22.54944136, 22.17400224, 22.57493527, ..., 20.64814469,
        21.08080793, 23.27907837],
       [27.71674995, 25.62207549, 25.15256705, ..., 26.49150667,
        23.30045887, 27.69182732],
       [22.25760503, 21.45900757, 20.47351905, ..., 21.43749075,
        20.38933425, 22.8621518 ]])

In [74]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 23.808194877324542)","(-7.82, 23.959019971572886)","(8.79, 22.60762540459238)","(99.0, 26.419619293473065)","(-8.16, 27.203804036174628)","(-7.52, 23.072614922752546)","(-8.5, 20.98098140589113)","(-9.85, 22.549722483160473)","(4.17, 26.170502563297095)","(-8.98, 24.816298698802772)","(-4.76, 21.343925426311554)","(-8.5, 26.635938104946216)","(-6.75, 22.137054039775357)","(-7.18, 24.56501700778128)","(8.45, 24.60310601746765)","(-7.18, 26.06486659058092)","(-7.52, 22.323897123716183)","(-7.43, 24.531454675349785)","(-9.81, 22.70011180409053)","(-9.85, 24.140294898528946)","(-9.85, 23.6124256742134)","(-9.37, 22.614268606284814)","(1.5, 23.58306553028485)","(-4.37, 21.080583050830636)","(99.0, 23.858907719918168)","(-8.5, 26.462215758696246)","(1.12, 23.150231227680013)","(7.82, 24.933257692870306)","(99.0, 23.927591010807276)","(9.13, 20.889458099500892)","(-7.43, 25.405088597619045)","(2.14, 22.626654573087315)","(-4.08, 26.165549531198934)","(-9.08, 23.801930521976747)","(7.82, 24.76538409678792)","(5.05, 24.5894536877208)","(4.95, 23.763170495781328)","(-9.17, 20.624708760998384)","(-8.4, 25.58441479196798)","(-8.4, 24.771209752506266)",...,"(8.59, 23.65848259637506)","(3.59, 23.60723558744544)","(-6.84, 23.66673862256551)","(-9.03, 21.890440635436136)","(2.82, 23.399119220752436)","(-1.36, 24.27904953112206)","(-9.08, 24.24768630583334)","(8.3, 24.420592173230464)","(5.68, 22.827154856949857)","(-4.81, 23.409387875418794)","(99.0, 25.261897708792887)","(99.0, 23.49763468800286)","(99.0, 23.194394060475723)","(99.0, 23.437828655390447)","(99.0, 26.528122895731574)","(99.0, 23.934372416534934)","(99.0, 23.055561086069872)","(99.0, 24.929867964801918)","(99.0, 23.30317038830347)","(99.0, 23.166671245178595)","(99.0, 25.168623269009572)","(-7.72, 24.951300977929204)","(99.0, 24.173508790757612)","(99.0, 26.139207086283)","(99.0, 22.190349330007418)","(99.0, 26.181714230818237)","(99.0, 25.088656246018342)","(99.0, 24.069170196491733)","(99.0, 23.621260790438814)","(99.0, 20.58699731997293)","(2.82, 22.03732799654398)","(99.0, 27.03729406889592)","(99.0, 21.246062542331842)","(99.0, 23.24839393122259)","(99.0, 24.960180366951374)","(99.0, 24.235135831147666)","(-5.63, 25.6600289429827)","(99.0, 22.721684543188456)","(99.0, 22.441371151122024)","(99.0, 22.322427865561764)"
1,"(100.0, 27.57873560572568)","(4.08, 24.954886382958477)","(-0.29, 26.352586922631563)","(99.0, 30.308066142867407)","(99.0, 27.596903029928296)","(-2.38, 25.259848399263817)","(-9.66, 26.22010114912067)","(99.0, 27.556101614226048)","(-5.34, 27.887728303427888)","(8.88, 28.713299213068723)","(9.22, 25.85693974142105)","(6.75, 28.43661924523272)","(8.64, 24.932634018203494)","(4.42, 25.583068293518732)","(7.43, 28.294101770429158)","(4.56, 27.47851289637195)","(-0.97, 24.41981476353971)","(4.66, 27.922238581044354)","(-0.68, 24.880694970515172)","(3.3, 28.160552771886902)","(-1.21, 24.104125729356724)","(0.87, 24.854104049456556)","(8.64, 26.674938800328267)","(8.35, 22.299631949149614)","(9.17, 27.382113414016988)","(0.05, 28.310306031901113)","(7.57, 25.60692044762333)","(4.71, 28.023763263533223)","(0.87, 28.622124177755616)","(-0.39, 25.651479039130347)","(6.99, 27.337332477074764)","(6.5, 25.01528813131257)","(-0.92, 26.776111053325277)","(7.14, 28.364317806552833)","(9.03, 27.50819093872185)","(-1.8, 25.993748542021876)","(0.73, 26.72441871544764)","(7.09, 26.005370681837054)","(3.4, 29.637559114392936)","(-0.87, 28.59980764070318)",...,"(-6.7, 27.031118357690072)","(-3.35, 26.65282273966971)","(-9.03, 26.90971736707492)","(4.47, 26.417682625451036)","(4.08, 26.211060147414916)","(-3.83, 28.060831676669146)","(8.74, 27.249795048965467)","(1.12, 27.373338051506632)","(0.78, 26.53770454212066)","(7.52, 27.3232401